In [1]:
import sys
sys.path.append("../../../../")
sys.path.append("../../")

In [4]:
!pip install -q --upgrade bitsandbytes==0.48.2 trl==0.25.1 peft

In [5]:
import os
import re
import torch
from torch import nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math 
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer , TrainingArguments , set_seed , BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig
from trl import SFTTrainer , SFTConfig
from datetime import datetime

In [6]:
BASE_MODEL = "meta-llama/Llama-3.2-3B"
PROJECT_NAME = "price-prediction"
USERNAME = "ujalaarshad17"
DATASET_NAME=f"{USERNAME}/items_prompts"
RUN_NAME =  f"{datetime.now():%Y-%m-%d_%H.%M.%S}"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
HUB_MODEL_NAME = f"{USERNAME}/{PROJECT_RUN_NAME}"

In [7]:
EPOCHS = 5
BATCH_SIZE = 64
MAX_SEQUENCE_LENGTH = 148
GRADIENT_ACCUMULATION_STEPS = 1

# QLORA 
QUANT_4_BIT = True
LORA_R = 32 
LORA_ALPHA = LORA_R * 2
ATTENTION_LAYERS = ["q_proj", "v_proj", "k_proj", "o_proj"]
TARGET_MODULES = ATTENTION_LAYERS 
LORA_DROPOUT = 0.1

# Hyperparameters
LEARNING_RATE = 1e-4
WARMUP_RATIO = 0.01
LR_SCHEDULER_TYPE = 'cosine'
WEIGHT_DECAY = 0.001
OPTIMIZER = "paged_adamw_32bit"

capability = torch.cuda.get_device_capability()
use_bf16 = capability[0] >= 8

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
VAL_SIZE = 500 
LOG_STEPS = 5 
SAVE_STEPS = 100 
LOG_TO_WANDB = True

In [ ]:
hf_token = ""
login(hf_token, add_to_git_credential=True)